In [100]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

! pip install mlxtend

# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here: 
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [101]:
# load the data set ans show the first five transaction
df = pd.read_csv("https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv")
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [102]:
print(set(np.ravel(df)))

{nan, 'Bread', 'Cheese', 'Eggs', 'Pencil', 'Milk', 'Wine', 'Bagel', 'Meat', 'Diaper'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [103]:
#create an itemset based on the products
itemset = set(items)

# encoding the feature
encoded_vals = []
for index, row in df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]

{nan: 0,
 'Milk': 0,
 'Bagel': 0,
 'Bread': 1,
 'Cheese': 1,
 'Eggs': 1,
 'Pencil': 1,
 'Wine': 1,
 'Meat': 1,
 'Diaper': 1}

In [104]:
  # create new dataframe from the encoded features
df2 = pd.DataFrame(encoded_vals)
  # show the new dataframe
df2.head()

,NaN,Milk,Bagel,Bread,Cheese,Eggs,Pencil,Wine,Meat,Diaper
0,0,0,0,1,1,1,1,1,1,1
1,0,1,0,1,1,0,1,1,1,1
2,1,1,0,0,1,1,0,1,1,0
3,1,1,0,0,1,1,0,1,1,0
4,1,0,0,0,0,0,1,1,1,0


In [105]:
df2.head()

,NaN,Milk,Bagel,Bread,Cheese,Eggs,Pencil,Wine,Meat,Diaper
0,0,0,0,1,1,1,1,1,1,1
1,0,1,0,1,1,0,1,1,1,1
2,1,1,0,0,1,1,0,1,1,0
3,1,1,0,0,1,1,0,1,1,0
4,1,0,0,0,0,0,1,1,1,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

In [106]:
df2 = df2.iloc[:, 1:]
df2.head()

,Milk,Bagel,Bread,Cheese,Eggs,Pencil,Wine,Meat,Diaper
0,0,0,1,1,1,1,1,1,1
1,1,0,1,1,0,1,1,1,1
2,1,0,0,1,1,0,1,1,0
3,1,0,0,1,1,0,1,1,0
4,0,0,0,0,0,1,1,1,0


## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products. 
For this case study, we will min_support=0.2

In [108]:
#Set threshold value untuk digunakan dalam penghitungan support
from mlxtend.frequent_patterns import apriori, association_rules
df_apriori = apriori(df2, min_support = 0.2, use_colnames = True, verbose = 1)
df_apriori

Processing 174 combinations | Sampling itemset size 3


C:\Users\User\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.501587,(Milk)
1,0.425397,(Bagel)
2,0.504762,(Bread)
3,0.501587,(Cheese)
4,0.438095,(Eggs)
5,0.361905,(Pencil)
6,0.438095,(Wine)
7,0.476190,(Meat)
8,0.406349,(Diaper)
9,0.225397,"(Milk, Bagel)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [109]:
df_ar = association_rules(df_apriori, metric = "confidence", min_threshold = 0.6)
df_ar

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
1,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
2,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650
3,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203
4,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754
5,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891
6,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540
7,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624
8,"(Milk, Cheese)",(Meat),0.304762,0.476190,0.203175,0.666667,1.400000,0.058050,1.571429
9,"(Milk, Meat)",(Cheese),0.244444,0.501587,0.203175,0.831169,1.657077,0.080564,2.952137
